In [1]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [2]:
# load RPE, PR, Recon3D, and basis Human1 models from cs mods folder
from pathlib import Path
from cobra.io import read_sbml_model, load_matlab_model
mod = read_sbml_model(Path().cwd() / 'cs_mods' / 'Human-GEM.xml' )
mod_RPE = read_sbml_model(Path().cwd() / 'cs_mods' / 'model_06_MeanExpression_RPE__VoigtEtAl2019_.xml')
mod_PR = read_sbml_model(Path().cwd() / 'cs_mods' / 'model_13_MeanExpression_RodPhotoreceptors__LiangEtAl_2019_.xml' )
mod_Recon3D = load_matlab_model(Path().cwd() / 'cs_mods' / 'Recon3D_model2_corrected_subsystems.mat' )

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


No defined compartments in model model2. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [3]:
# modify context specific models
from src.modify_model import add_all_EX_rxns, remove_compartment

for m in [mod_RPE, mod_PR]: # add ATP DM rxn and lost EX rxns to context specific models
    m.add_reactions([mod.reactions.get_by_id('MAR03964').copy()]) # make sure it contains ATP DM rxn
    m = add_all_EX_rxns(mod,m) # add lost EX rxns
    for r in [r for r in m.reactions if len(r.products)==0]: 
        r.bounds = (0,1000) # close lb for exchange reactions

mod_PR = remove_compartment(mod_PR.copy(), 'n')  # remove nucleus ('n') from PR model
mod_no_nucleus = remove_compartment(mod.copy(), 'n')  # remove nucleus ('n') from Human1 model
mod_Recon3D_no_nucleus = remove_compartment(mod_Recon3D.copy(), 'n')  # remove nucleus ('n') from Human1 model

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpnwg30l98.lp
Reading time = 0.08 seconds
: 5013 rows, 12490 columns, 46478 nonzeros


C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpc1tadjwo.lp
Reading time = 0.18 seconds
: 8369 rows, 26140 columns, 111668 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp964npv__.lp
Reading time = 0.26 seconds
: 5835 rows, 21200 columns, 80850 nonzeros


In [4]:
# make combined RPE_PR models
from src.combine_RPE_PR import make_RPE_PR_model, make_RPE_PR_model_Recon3D
mod_RPE_PR = make_RPE_PR_model(mod_RPE.copy(), mod_PR.copy())
mod_Human1_Human1 = make_RPE_PR_model(mod.copy(), mod_no_nucleus.copy())
mod_Recon3D_Recon3D = make_RPE_PR_model_Recon3D(mod_Recon3D.copy(), mod_Recon3D_no_nucleus.copy())

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpcvlmgtmy.lp
Reading time = 0.09 seconds
: 5517 rows, 14194 columns, 54382 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpchm0s1cu.lp
Reading time = 0.08 seconds
: 4921 rows, 12234 columns, 45312 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp7jk4k53u.lp
Reading time = 0.12 seconds
: 7182 rows, 19688 columns, 65842 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpum_hdmkp.lp
Reading time = 0.08 seconds
: 4921 rows, 12234 columns, 45312 nonzeros


C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpy9rpq2h8.lp
Reading time = 0.18 seconds
: 8369 rows, 26140 columns, 111668 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmppcskypuo.lp
Reading time = 0.16 seconds
: 8185 rows, 25654 columns, 109522 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp7ne4pvlu.lp
Reading time = 0.09 seconds
: 10034 rows, 36120 columns, 143572 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpvvm3brtg.lp
Reading time = 0.08 seconds
: 8185 rows, 25654 columns, 109522 nonzeros


C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp7c94vzt4.lp
Reading time = 0.04 seconds
: 5835 rows, 21200 columns, 80850 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp47n3lf0m.lp
Reading time = 0.04 seconds
: 5740 rows, 20858 columns, 79758 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpd72we3iy.lp
Reading time = 0.06 seconds
: 7414 rows, 30658 columns, 111178 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp_9si5o_t.lp
Reading time = 0.05 seconds
: 5740 rows, 20858 columns, 79758 nonzeros


C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [5]:
from cobra.io import write_sbml_model
# save combined RPE_PR models
write_sbml_model(mod_RPE_PR, "mod_RPE_PR.xml")
write_sbml_model(mod_Human1_Human1, "mod_Human1_Human1.xml")
write_sbml_model(mod_Recon3D_Recon3D, "mod_Recon3D_Recon3D.xml")

Error encountered trying to <Setting notes on sbase: <Reaction R_ADCim_RPE "Acetoacetate Decarboxylation (Irreversible), Mitochondrial">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_CHOLtu_RPE "Choline Uniport">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_DEBRISOQUINEt_RPE "Debrisoquine Uniport">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a